# 6.390 Spring 2023 Homework 3

**If you haven't already, please hit :**

`File` -> `Save a Copy in Drive`

**to copy this notebook to your Google drive, and work on a copy. If you don't do this, your changes won't be saved!**

In [ ]:
# Run this cell to download the test functions for HW 3
!rm -f hw03_tests.py
!wget --quiet --no-check-certificate https://introml.mit.edu/_static/spring23/homework/hw03/hw03_tests.py
from hw03_tests import *

import numpy as np

#### 10.2 Finding the Best Parameters (Optional)

Let's load the Boston Housing dataset. Our goal is to build a linear regression model (with regularization) to predict the TARGET_VAL (which is the median value of owner-occupied homes) using all other available features in the dataset.

For more information about the Boston housing dataset, please visit this [link](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html).

Note that the data pre-processing routine below normalizes each feature. You will learn more about Feature transformations in Week 5.

In what follows, we use Cross-Validation to select the best hyperparameters for gradient descent on the ridge regression model. Using the best hyperparameters, we will then make predictions on a reserved test set. You will also compare the results when using the gradient descent based implementation vs the analytic (closed form) solution.

In [ ]:
## DO NOT EDIT BELOW.
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold

# Pre-Processing the data and returning the train and test sets.

# load the dataset and do some data exploration
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

raw_data = np.concatenate((data, target[:, None]), axis=1)
xvars = [
    "CRIM",
    "ZN",
    "INDUS",
    "CHAS",
    "NOX",
    "RM",
    "AGE",
    "DIS",
    "RAD",
    "TAX",
    "PTRATIO",
    "B",
    "LSTAT",
]
yvars = ["TARGET_VAL"]

data = pd.DataFrame(raw_data, columns=xvars + yvars)

# Get the train and test splits to be used later.
X_train, y_train, X_test, y_test = get_data_splits_with_transforms(data, xvars, yvars)

**CODE REQUIRED HERE** Before we start using the Boston Housing dataset, let's implement the `ridge_gd` function. Given an input `X_train`, `y_train`, `lam`, `theta`, `step_size_fn`, and `num_steps`, run gradient descent on `X_train` and `y_train` starting from `theta = (dx1) vector of zeros`. Return the value of $\theta$ after running `num_steps` iterations of gradient descent.

```
inputs:
  X_train: a dxn numpy array
  y_train: a 1xn numpy array
  lam: lambda
  step_size_fn: a function that takes in i, the current training iteration, and returns the step size for iteration i
  num_steps: number of iterations

outputs:
  theta: value of theta after num_steps iterations of gradient descent
```
**Hint**: Your implementations of `objective_func` and `objective_func_grad` are very useful here! \
**Hint**: You can also use your `gd` function \
**Hint**: Previously, you've minimized f as a function of x. Now, X and y are constant. What variable are you minimizing over now? \

In [ ]:
def ridge_gd(X_train, y_train, lam, step_size_fn, num_steps=2000):
    # TODO
    # hint: start from theta = (dx1) vector of zeros

In [ ]:
TestRidgeGD(ridge_gd)

**CODE REQUIRED HERE** In `cross_validate_gd`, run 5-fold cross-validation on the X and y dataset. Use gradient descent to train a linear model for the `X`, `y` data. We've provided a for loop that iterates over each split. In this code:

```
  X_train_split, y_train_split: data to use for training. This is a (d x n) numpy array, where n=the number of datapoints in k-1 folds
  X_val_split, y_val_split: data to use for evaluating the model. This is a (d x n) numpy array, where n=the number of datapoints in 1 fold
```

**Hint**: Use `ridge_gd` here. \
**Hint**: Take a look at the solutions for last week's cross_validate code if you get stuck

In [ ]:
def cross_validate_gd(X, y, lam, step_size_fn):
    """
    Returns k-fold cross-validation loss. On each of the k folds,
    train a linear regression model using gradient descent. Return
    the average loss across the k folds.
    """
    total_loss = 0
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(X, y=y):
        X_train_split, y_train_split = X[train_index].T, y[train_index].T
        # TODO - train model on X_train_split, y_train_split using gradient descent
        # hint - use variables step_size_fn and lam
        # TODO - evaluate model on X_val_split, y_val_split, add loss to total_loss
    return total_loss / kf.n_splits

Now it's time to run grid search! We are interested in running grid search over $\lambda \in \{{1e-4, 1e-3, \cdots, 1e-1\}}$ and $\eta \in \{{1e-6, 1e-5, \cdots, 1e-2\}}$.

These two cells are ready to run if you've correctly implemented `cross_validate_gd`. Use the outputs of these cells to answer the rest of problem 5.2.

We've also already implemented `cross_validate_analytic` for you. This function returns the cross-validation loss for linear regression models trained with the analytic solution for the squared loss equation.

**Note: The next two cells print the cross-validation loss, not the testing set loss! Run the last cell in this notebook for the testing set loss.**


In [ ]:
learning_rates = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
lams = [1e-1, 1e-2, 1e-3, 1e-4]

# This code runs grid search over the training parameters in `learning_rates` and `lams`
for rate in learning_rates:
    for lam in lams:
        learning_rate_fn = lambda i: rate  # learning rate = `rate` throughout training
        cross_validation_loss = cross_validate_gd(
            X_train, y_train, lam, learning_rate_fn
        )
        print(
            f"Loss on dataset with lambda={lam}, rate={rate} : cross_validation_loss {cross_validation_loss:.6f}"
        )

In [ ]:
lams = [1e-1, 1e-2, 1e-3, 1e-4]

# This code runs grid search over the training parameters in `lams`
for lam in lams:
    cross_validation_loss = cross_validate_analytic(X_train, y_train, lam).item()
    print(
        f"Loss on dataset with lambda={lam}: cross_validation_loss {cross_validation_loss:.6f}"
    )

We will now use the best params found above to build a model on the entire training set (X_train, y_train), get the $\theta$ values and use them to make predictions for the test set (X_test) and evaluate the error using the *actual* values (y_test). We will compare this error for the gradient descent based implementation vs the analytic solution.


**CODE REQUIRED HERE**:

1. Update **best_lam_gd** and **best_rate_gd** using the best $\lambda$ and $\eta$ values you found using **cross_validate_gd**() above.

2. Update **best_lam_analytic** using the best $\lambda$ value found by using **cross_validate_analytic**() above.



In [ ]:
## DO NOT EDIT THESE FUNCTIONS.
# Returns the test set predictions and error for the GD based implementation.
def get_gd_predictions_and_error(
    objective_func,
    objective_func_grad,
    gd_func,
    X_train,
    y_train,
    X_test,
    y_test,
    best_lam_gd,
    best_rate_gd,
):
    num_steps = 5000
    theta_gd = ridge_gd(
        X_train.T, y_train.T, best_lam_gd, lambda i: best_rate_gd, num_steps=num_steps
    )
    gd_predictions = theta_gd.T @ X_test.T
    gd_error = np.mean((gd_predictions - y_test) ** 2)
    return gd_predictions, gd_error


# Returns the test set predictions and error using the Analytic expression.
def get_analytic_predictions_and_error(X_train, y_train, X_test, y_test, best_lam):
    theta_analytic = ridge_analytic(X_train.T, y_train.T, best_lam)
    analytic_predictions = theta_analytic.T @ X_test.T
    analytic_error = np.mean((analytic_predictions - y_test) ** 2)
    return analytic_predictions, analytic_error

In [ ]:
#### Using the functions above along with the best performing hyperparams to
### determine the test set errors. Please specify the best lambda and learning
### rates for the GD and Analytic cases that you found above.

# GD
best_lam_gd = None  ### TODO: to be specified
best_rate_gd = None  ### TODO: to be specified

# get_gd_predictions() function is defined in the hw03 code you imported at
# the very top. Check the code out if you are curious about the implementation.
gd_predictions, gd_error = get_gd_predictions_and_error(
    objective_func,
    objective_func_grad,
    gd,
    X_train,
    y_train,
    X_test,
    y_test,
    best_lam_gd,
    best_rate_gd,
)

# Analytic
best_lam_analytic = None  ### TODO: to be specified

# get_analytic_predictions_and_error() function is defined in the hw03 code
# you imported at the very top. Check the code out if you are curious about the
# implementation.
analytic_predictions, analytic_error = get_analytic_predictions_and_error(
    X_train, y_train, X_test, y_test, best_lam_analytic
)


print(f"Test loss for GD based implementation={gd_error:0.3f}")
print(f"Test loss for Analytic (closed form) implementation={analytic_error:0.3f}")


#### (Optional) Compare the results by viewing the scatter plots for predictions.
plt.scatter(y_test, gd_predictions, color="red", label="GD")
plt.scatter(y_test, analytic_predictions, color="blue", label="Analytic")
plt.xlabel("Actual")
plt.ylabel("Predictions")
plt.title("Predictions vs Actual Scatter Plot")
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.legend(loc="upper right")
plt.show()